In [1]:
import requests
import requests_cache
from bs4 import BeautifulSoup
import time
import json
import re
import pandas as pd # Dùng để đọc và hiển thị file csv/tsv
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from requests_html import HTML

________________________________________

**Bấm để đồng ý cookies**

In [2]:
# Đồng ý cookies
def accept_cookies(browser):
    cookies = browser.find_element(By.XPATH,"//button[@id='onetrust-accept-btn-handler']")
    cookies.click()

**Kéo xuống đến cuối trang với số lần dựa theo limit (số playlists muốn parse tối đa)**

In [3]:
def scroll_down(browser, limit):
    # số tracks hiện lên mỗi lần scroll đến cuối trang
    tracks_per_scroll = 10
    # số lần scroll xuống cuối trang
    scrolls = limit // tracks_per_scroll
    for i in range(scrolls - 1):
        browser.execute_script("window.scrollTo(0, document.body.scrollHeight)")
        time.sleep(3)

**Lấy tất cả users và tracks**

In [4]:
def get_all_users_tracks(soup):
    songs = soup.find_all("li", "compactTrackList__item")
    for song in songs:
        # lấy user của track
        user = song.find("span", "compactTrackListItem__user")
        # kiểm tra nếu không có text
        if user is not None:
            userTitle = user.get_text().replace("-","").strip()
            users.add(userTitle)
                
        # lấy track của playlist
        track = song.find("span", "compactTrackListItem__trackTitle")
        # kiểm tra nếu không có text
        if track is not None:
            trackTitle = track.get_text().strip()
            # kiểm tra bị trùng
            if trackTitle not in tracks["track"]:
                tracks["track"].append(trackTitle)
                # kiểm tra nếu track không có user ở kế bên thì lấy user của playlist
                userTitle = soup.find("span", "soundTitle__usernameText").get_text().strip() if (user is None) else user.get_text().replace("-","").strip()
                tracks["user"].append(userTitle)

**Lấy tất cả thông tin (users, tracks, playlists)**

In [10]:
def get_all(browser):
    soup = BeautifulSoup(browser.page_source)
    # trích xuất playlist
    posts = soup.find_all("li", "searchList__item")
    for post in posts:
        # lấy user của playlist
        user = post.find("span", "soundTitle__usernameText").get_text().strip()
        users.add(user)
        playlists["user"].append(user)
        
        # lấy tên playlist
        playlist = post.find("a", "soundTitle__title").get_text().strip()
        playlists["playlist"].append(playlist)
        
        # lấy tất cả tracks và users có trong playlist
        get_all_users_tracks(post)
        
        # lấy Id của các tracks có trong playlist theo index
        track_list = post.find_all("span", "compactTrackListItem__trackTitle")
        # lưu index của track
        trackIds = []
        for item in track_list:
            # lấy index của track
            trackIds.append(tracks["track"].index(item.get_text().strip()))
        playlists["tracks"].append(trackIds)

**Chuyển từ list sang csv/tsv**

In [6]:
def list_to_tsv_csv(listItem, outputFile):
    df = pd.DataFrame(listItem)
    df.to_csv(outputFile, sep='\t')

**Main**

In [7]:
def main(limit):
    alphabet = ['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z']
    # lặp cho từng chữ cái
    for letter in alphabet:
        # mỗi chữ cái sẽ mở 1 browser Chrome mới, sau khi lấy hết xong thì tắt.
        url = "https://soundcloud.com/search/sets?q=" + letter
        browser = webdriver.Chrome()
        browser.get(url)
        
        # đồng ý cookies
#         time.sleep(5)
#         accept_cookies(browser)
        
        # scroll xuống theo limit cho phép
        time.sleep(1)
        scroll_down(browser, limit)
        
        # lấy hết tất cả (user, track, playlist)
        time.sleep(1)
        get_all(browser)
        
        # tắt browser
        time.sleep(1)
        browser.quit()

**Khởi tạo các list để lưu thông tin**

In [8]:
# Lưu user
users = set()
# Lưu track
tracks = {"track":[], "user":[]}
# Lưu playlist
playlists = {"playlist":[], "user":[], "tracks":[]}

In [11]:
# giới hạn lấy playlist cho các chữ cái
limit = 10
main(limit)

WebDriverException: Message: disconnected: Unable to receive message from renderer
  (Session info: chrome=95.0.4638.69)
Stacktrace:
0   chromedriver                        0x000000010536ec99 __gxx_personality_v0 + 579209
1   chromedriver                        0x00000001052fa5f3 __gxx_personality_v0 + 102371
2   chromedriver                        0x0000000104ec0288 chromedriver + 172680
3   chromedriver                        0x0000000104eab7c1 chromedriver + 88001
4   chromedriver                        0x0000000104eaafd3 chromedriver + 85971
5   chromedriver                        0x0000000104ec6bfa chromedriver + 199674
6   chromedriver                        0x0000000104f252c1 chromedriver + 586433
7   chromedriver                        0x0000000104f12693 chromedriver + 509587
8   chromedriver                        0x0000000104ee8b7e chromedriver + 338814
9   chromedriver                        0x0000000104ee9ea5 chromedriver + 343717
10  chromedriver                        0x0

**Lưu data vào file tsv**

In [ ]:
outputFile = "Crawl_data/user1.csv"
list_to_tsv_csv(users, outputFile)
outputFile = "Crawl_data/track1.csv"
list_to_tsv_csv(tracks, outputFile)
outputFile = "Crawl_data/playlist1.csv"
list_to_tsv_csv(playlists, outputFile)

In [12]:
print(len(users))
print(len(tracks["user"]))
print(len(playlists["playlist"]))

84
82
20


In [22]:
playlists["tracks"]

[[0, 1, 2, 3, 4],
 [5],
 [6, 7, 8, 9, 10],
 [11, 12, 13, 14, 15],
 [16, 17, 18, 19, 20],
 [21, 22, 23, 24, 25],
 [26, 27, 28, 29, 30],
 [31, 32, 33, 34, 35],
 [36, 37, 38, 39, 40],
 [41, 42, 43, 44, 45],
 [46, 47, 48, 49, 50],
 [],
 [51, 52, 53, 54, 55],
 [],
 [56, 57, 58, 59, 60],
 [61, 62, 63, 64, 65],
 [66, 67, 68, 69, 70],
 [71, 72, 73, 74, 75],
 [76, 77],
 [78, 79, 80, 81]]